<img src="https://www.iscte-iul.pt/assets/images/logo_iscte_detailed.svg" style="width: 450px;margin-top:30px;" align ="center">

<div style= "font-size: 35px;  margin-top:40px; font-weight:bold; font-family: 'Avenir Next LT Pro', sans-serif;"><center>User Clustering: <strong>E-Commerce</strong></center></div>
<div style= "font-size: 30px; font-weight:bold; font-family: 'Avenir Next LT Pro', sans-serif;"><center>Organizing all variables and data to the models phase</center></div>

<div style= "font-size: 27px;font-weight:bold;line-height: 1.1; margin-top:40px; font-family: 'Avenir Next LT Pro', sans-serif;"><center>Processamento e Modelação de Big Data 2024/2025</center></div> <br>

   <div style= "font-size: 20px;font-weight:bold; font-family: 'Avenir Next LT Pro', sans-serif;"><center> Grupo 7:</center></div>
   <div><center> Diogo Freitas | 104841 </center></div>
   <div><center> João Francisco Botas | 104782 </center></div>
   <div><center> Miguel Gonçalves | 105944 </center></div>
   <div><center> Ricardo Galvão | 105285 </center></div>

In [1]:
# Importações Básicas
import pyspark
import pandas as pd
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.functions import split, isnan, col, isnull, avg, count, round
from pyspark.sql.types import (
    StructType, StructField,
    StringType, LongType, DoubleType, TimestampType
)


# Create a Spark session
spark = SparkSession.builder \
    .appName("Projeto") \
    .getOrCreate()

In [3]:
# Criação da Sessão
spark = SparkSession.builder.appName("Projeto").getOrCreate()

# Leitura de dados
data_dir = '../data/'
file_ec = data_dir + 'user_df.parquet'

# Carrega as primeiras 10.000 linhas
ec_user = spark.read.parquet(file_ec)

In [4]:
ec_user.show()

+---------+-------------+-----------+-----------+----------+---------+---------+-------+------------+----+----+-----+-----------+
|  user_id|average_price|total_views|electronics|appliances|furniture|computers|apparel|construction|auto|kids|sport|accessories|
+---------+-------------+-----------+-----------+----------+---------+---------+-------+------------+----+----+-----+-----------+
|515761354|       141.55|          3|          0|         3|        0|        0|      0|           0|   0|   0|    0|          0|
|565865924|       949.47|          1|          1|         0|        0|        0|      0|           0|   0|   0|    0|          0|
|527322328|      197.797|          7|          0|         7|        0|        0|      0|           0|   0|   0|    0|          0|
|553334193|       172.23|          1|          1|         0|        0|        0|      0|           0|   0|   0|    0|          0|
|565921426|       165.26|          1|          1|         0|        0|        0|      0|  

## KMeans Clustering

In [18]:
feature_cols = ec_user.columns
feature_cols.pop(0)

'user_id'

In [19]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
client_features = assembler.transform(ec_user)

In [22]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator


# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(client_features)

# Make predictions
predictions = model.transform(client_features)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.8645130956054886
Cluster Centers: 
[2.31897798e+02 4.23121387e+00 1.88191577e+00 7.58876961e-01
 4.99587118e-01 2.55986788e-01 3.64161850e-01 1.61849711e-01
 1.19735756e-01 1.03220479e-01 3.30305533e-02 5.20231214e-02]
[1.07443439e+03 3.89641434e+00 2.55776892e+00 1.27490040e-01
 1.75298805e-01 8.84462151e-01 7.96812749e-03 3.98406375e-03
 0.00000000e+00 0.00000000e+00 1.39442231e-01 0.00000000e+00]


In [38]:
predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  251|
|         0| 1211|
+----------+-----+



In [33]:
cluster_profiles = predictions.groupBy("prediction").mean()

In [34]:
for c in cluster_profiles.columns:
    cluster_profiles = cluster_profiles.withColumn(c, F.round(c, 2))

In [35]:
cluster_profiles.show()

+----------+--------------+------------------+----------------+----------------+---------------+--------------+--------------+------------+-----------------+---------+---------+----------+----------------+---------------+
|prediction|  avg(user_id)|avg(average_price)|avg(total_views)|avg(electronics)|avg(appliances)|avg(furniture)|avg(computers)|avg(apparel)|avg(construction)|avg(auto)|avg(kids)|avg(sport)|avg(accessories)|avg(prediction)|
+----------+--------------+------------------+----------------+----------------+---------------+--------------+--------------+------------+-----------------+---------+---------+----------+----------------+---------------+
|         1| 5.376852892E8|           1074.43|             3.9|            2.56|           0.13|          0.18|          0.88|        0.01|              0.0|      0.0|      0.0|      0.14|             0.0|            1.0|
|         0|5.4016818249E8|             231.9|            4.23|            1.88|           0.76|           0.5| 